# W12 — Statystyka: rozkłady i testy hipotez

**Programowanie w Pythonie II** | Politechnika Opolska  
**Temat:** Rozkład normalny, t-test, A/B testing, chi-kwadrat, przedziały ufności

---

> *"Statystyka to narzędzie decyzyjne — nie matematyka dla matematyki."*

### Co dziś robimy:
1. Rozkład normalny — `scipy.stats.norm`, PDF, CDF, kwantyle
2. Diagnostyka normalności — QQ-plot, test Shapiro-Wilka
3. T-test — jednorodkowy, niezależny (Welch), sparowany
4. Pełna analiza A/B testu kampanii marketingowej
5. Test chi-kwadrat dla danych kategorycznych
6. Przedziały ufności — ocena niepewności estymacji

## Setup

In [ ]:
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

np.random.seed(42)
sns.set_theme(style='whitegrid', palette='muted')
print(f"scipy: {__import__('scipy').__version__}")
print(f"numpy: {np.__version__}")
print("Środowisko gotowe.")

---
## 1. Rozkład normalny — scipy.stats.norm

### 1.1 PDF i CDF — dwa spojrzenia na ten sam rozkład

Pracujemy z rozkładem wzrostu dorosłych Polaków:
- **μ = 170 cm** (średnia)
- **σ = 8 cm** (odchylenie standardowe)

Trzy kluczowe funkcje z `scipy.stats.norm`:
- `pdf(x)` — gęstość: jak gęsto są obserwacje w okolicach x
- `cdf(x)` — dystrybuanta: P(X ≤ x)
- `ppf(p)` — kwantyl: dla jakiego x zachodzi P(X ≤ x) = p

In [ ]:
np.random.seed(42)

mu = 170      # średni wzrost dorosłego Polaka (cm)
sigma = 8     # odchylenie standardowe

x = np.linspace(mu - 4*sigma, mu + 4*sigma, 300)

pdf = stats.norm.pdf(x, loc=mu, scale=sigma)
cdf = stats.norm.cdf(x, loc=mu, scale=sigma)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# PDF
axes[0].plot(x, pdf, 'b-', linewidth=2.5, label='PDF')
axes[0].axvline(mu, color='red', linestyle='--', linewidth=1.5, label=f'μ = {mu}')
axes[0].fill_between(
    x, pdf,
    where=(x >= mu - sigma) & (x <= mu + sigma),
    alpha=0.3, color='blue', label='68% (±1σ)'
)
axes[0].fill_between(
    x, pdf,
    where=(x >= mu - 2*sigma) & (x <= mu + 2*sigma),
    alpha=0.15, color='green', label='95% (±2σ)'
)
axes[0].set_title('PDF — funkcja gęstości prawdopodobieństwa\n(wzrost dorosłych Polaków)', fontsize=11)
axes[0].set_xlabel('Wzrost (cm)')
axes[0].set_ylabel('Gęstość prawdopodobieństwa')
axes[0].legend(fontsize=9)

# CDF
axes[1].plot(x, cdf, 'g-', linewidth=2.5)
axes[1].axhline(0.5, color='red', linestyle='--', linewidth=1.2, label='50. percentyl (mediana)')
axes[1].axhline(0.95, color='orange', linestyle=':', linewidth=1.2, label='95. percentyl')
axes[1].set_title('CDF — dystrybuanta\n(wzrost dorosłych Polaków)', fontsize=11)
axes[1].set_xlabel('Wzrost (cm)')
axes[1].set_ylabel('Skumulowane prawdopodobieństwo')
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()
plt.close()

### 1.2 Praktyczne obliczenia biznesowe

In [ ]:
mu, sigma = 170, 8

# PYTANIE 1: jaki % populacji ma wzrost > 185 cm?
prob_above_185 = 1 - stats.norm.cdf(185, loc=mu, scale=sigma)
print(f"Wzrost > 185 cm: {prob_above_185*100:.1f}% populacji")

# PYTANIE 2: granica górnych 10% (dla produkcji ubrań XL/XXL)
percentyl_90 = stats.norm.ppf(0.90, loc=mu, scale=sigma)
print(f"Górne 10% populacji: wzrost > {percentyl_90:.1f} cm")

# PYTANIE 3: % populacji o wzroście 160–180 cm
prob_160_180 = stats.norm.cdf(180, mu, sigma) - stats.norm.cdf(160, mu, sigma)
print(f"Wzrost 160–180 cm: {prob_160_180*100:.1f}% populacji")

print()
# PYTANIE 4 — e-commerce: czas ładowania strony ~ N(2.5s, 0.4s)
mu_load, sigma_load = 2.5, 0.4
prob_slow = 1 - stats.norm.cdf(3.0, loc=mu_load, scale=sigma_load)
sla_99 = stats.norm.ppf(0.99, loc=mu_load, scale=sigma_load)
print(f"Czas ładowania > 3s (odejście użytkownika): {prob_slow*100:.1f}% sesji")
print(f"SLA 99%: 99% stron ładuje się poniżej {sla_99:.2f}s")

---
## 2. Diagnostyka normalności — QQ-plot i Shapiro-Wilk

Zanim użyjemy testu statystycznego (np. t-testu), sprawdzamy czy dane są normalne.

Dwa narzędzia:
- **QQ-plot** (visualny) — punkty blisko linii = dane normalne
- **Test Shapiro-Wilka** (formalny) — H₀: dane są normalne; p > 0.05 = nie odrzucamy H₀

In [ ]:
np.random.seed(42)

# Generujemy próbkę — wzrost 200 klientów sklepu odzieżowego
wzrost_klientow = np.random.normal(loc=170, scale=8, size=200)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Histogram vs krzywa teoretyczna
axes[0].hist(wzrost_klientow, bins=25, density=True,
             alpha=0.7, color='steelblue', edgecolor='white',
             label='Dane (n=200)')
x_range = np.linspace(wzrost_klientow.min(), wzrost_klientow.max(), 200)
axes[0].plot(
    x_range,
    stats.norm.pdf(x_range, loc=wzrost_klientow.mean(), scale=wzrost_klientow.std()),
    'r-', linewidth=2.5,
    label=f'N(μ={wzrost_klientow.mean():.1f}, σ={wzrost_klientow.std():.1f})'
)
axes[0].set_title('Histogram vs rozkład normalny\n(wzrost klientów)', fontsize=11)
axes[0].set_xlabel('Wzrost (cm)')
axes[0].set_ylabel('Gęstość')
axes[0].legend()

# QQ-plot
stats.probplot(wzrost_klientow, dist='norm', plot=axes[1])
axes[1].set_title('QQ-plot — czy dane są normalne?\n(punkty blisko linii = normalność)', fontsize=11)
axes[1].get_lines()[1].set_color('red')
axes[1].get_lines()[1].set_linewidth(2)

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
np.random.seed(42)

# Test Shapiro-Wilka — formalny test normalności
dane_normalne     = np.random.normal(170, 8, 100)
dane_asymetryczne = np.random.exponential(scale=20, size=100) + 5
czasy_obslugi     = np.abs(np.random.normal(5, 2, 100)) + 1

print(f"{'Zbiór danych':40s} | {'W':>8} | {'p':>8} | Wniosek")
print("-" * 75)
for nazwa, dane in [
    ('Dane normalne (wzrost)', dane_normalne),
    ('Dane asymetryczne (zarobki)', dane_asymetryczne),
    ('Czasy obsługi (realne)', czasy_obslugi)
]:
    stat, p = stats.shapiro(dane)
    normalnosc = "NORMALNY  ✓" if p > 0.05 else "NIE-normalny ✗"
    print(f"{nazwa:40s} | {stat:>8.4f} | {p:>8.4f} | {normalnosc}")

print()
print("H₀: dane pochodzą z rozkładu normalnego")
print("p > 0.05 → nie odrzucamy H₀ → dane są zgodne z normalnym")
print("p ≤ 0.05 → odrzucamy H₀ → dane NIE są normalne")

In [ ]:
np.random.seed(42)

# Wizualizacja: trzy typy rozkładów — histogram i QQ-plot
datasets = [
    ('Wzrost klientów\n(normalny)', np.random.normal(170, 8, 200)),
    ('Zarobki\n(asymetryczny)', np.random.exponential(20, 200) * 1000 + 2000),
    ('Opóźnienia dostaw\n(długi ogon)', np.random.gamma(2, 3, 200)),
]

fig, axes = plt.subplots(2, 3, figsize=(14, 8), constrained_layout=True)

for col, (tytul, dane) in enumerate(datasets):
    # Histogram + KDE
    axes[0, col].hist(dane, bins=25, density=True, alpha=0.6,
                      color='steelblue', edgecolor='white')
    x_k = np.linspace(dane.min(), dane.max(), 200)
    kde = stats.gaussian_kde(dane)
    axes[0, col].plot(x_k, kde(x_k), 'r-', linewidth=2)
    stat, p = stats.shapiro(dane)
    axes[0, col].set_title(f'{tytul}\nShapiro p={p:.4f}', fontsize=10)
    axes[0, col].set_ylabel('Gęstość' if col == 0 else '')

    # QQ-plot
    stats.probplot(dane, dist='norm', plot=axes[1, col])
    axes[1, col].set_title('')
    axes[1, col].set_xlabel('Kwantyle teoretyczne')
    if col > 0:
        axes[1, col].set_ylabel('')

axes[0, 0].set_ylabel('Gęstość')
axes[1, 0].set_ylabel('Kwantyle empiryczne')

fig.suptitle('Rozkłady: normalny vs asymetryczny — histogram i QQ-plot',
             fontsize=13, fontweight='bold')
plt.show()
plt.close()

---
## 3. T-test — trzy scenariusze biznesowe

| Scenariusz | Test | Funkcja |
|-----------|------|---------|
| Jedna próba vs wartość oczekiwana | Jednorodkowy | `ttest_1samp(dane, popmean=X)` |
| Dwie różne grupy | Welch (niezależny) | `ttest_ind(A, B, equal_var=False)` |
| Te same osoby, dwa pomiary | Sparowany | `ttest_rel(przed, po)` |

### 3.1 T-test jednorodkowy — czy spełniamy normę?

In [ ]:
np.random.seed(42)

# SCENARIUSZ: Dział QA twierdzi że czas obsługi = 5.0 min.
# Mierzymy 50 faktycznych obsług. Czy deklaracja jest prawdziwa?
czasy_obslugi = np.random.normal(loc=5.8, scale=1.2, size=50)

print("=" * 50)
print("T-test jednorodkowy — czas obsługi klienta")
print("=" * 50)
print(f"H₀: μ = 5.0 min (QA twierdzi)")
print(f"H₁: μ ≠ 5.0 min")
print()
print(f"Dane: n={len(czasy_obslugi)}, x̄={czasy_obslugi.mean():.2f} min, s={czasy_obslugi.std():.2f} min")
print()

t_stat, p_val = stats.ttest_1samp(czasy_obslugi, popmean=5.0)

print(f"Statystyka t  = {t_stat:.4f}")
print(f"p-wartość     = {p_val:.4f}")
print()
if p_val < 0.05:
    print("WYNIK: p < 0.05 → Odrzucamy H₀")
    print("WNIOSEK: Czas obsługi STATYSTYCZNIE ISTOTNIE różni się od 5 minut.")
    print(f"Faktyczna średnia ({czasy_obslugi.mean():.1f} min) jest wyższa niż deklarowane 5 min.")
    print("Akcja: Rewizja normy lub usprawnienie procesu obsługi.")
else:
    print("WYNIK: p ≥ 0.05 → Brak podstaw do odrzucenia H₀")
    print("WNIOSEK: Brak statystycznych dowodów, że czas obsługi różni się od 5 min.")

### 3.2 T-test niezależnych grup — A/B test wartości koszyka

In [ ]:
np.random.seed(42)

# SCENARIUSZ: A/B test strony produktowej — wartość koszyka (PLN)
koszyk_A = np.random.normal(loc=250, scale=60, size=80)   # stara strona
koszyk_B = np.random.normal(loc=270, scale=65, size=80)   # nowa strona

print("=" * 55)
print("T-test niezależnych grup (A/B test koszyka)")
print("=" * 55)
print(f"H₀: μ_A = μ_B")
print(f"H₁: μ_A ≠ μ_B")
print()
print(f"Koszyk A: n={len(koszyk_A)}, x̄={koszyk_A.mean():.2f} PLN, s={koszyk_A.std():.2f} PLN")
print(f"Koszyk B: n={len(koszyk_B)}, x̄={koszyk_B.mean():.2f} PLN, s={koszyk_B.std():.2f} PLN")
print(f"Różnica:  {koszyk_B.mean() - koszyk_A.mean():.2f} PLN")
print()

# equal_var=False → Welch's t-test (bezpieczniejszy, nie zakłada równej wariancji)
t_stat, p_val = stats.ttest_ind(koszyk_A, koszyk_B, equal_var=False)

# Przedział ufności dla różnicy
n1, n2 = len(koszyk_A), len(koszyk_B)
mean_diff = koszyk_B.mean() - koszyk_A.mean()
se_diff = np.sqrt(koszyk_A.var()/n1 + koszyk_B.var()/n2)
df_approx = n1 + n2 - 2
ci_low = mean_diff - stats.t.ppf(0.975, df_approx) * se_diff
ci_high = mean_diff + stats.t.ppf(0.975, df_approx) * se_diff

print(f"Statystyka t  = {t_stat:.4f}")
print(f"p-wartość     = {p_val:.4f}")
print(f"95% CI diff   = [{ci_low:.2f}, {ci_high:.2f}] PLN")
print()
if p_val < 0.05:
    print(f"WYNIK: p < 0.05 → Odrzucamy H₀")
    print(f"WNIOSEK: Nowa strona ISTOTNIE zmienia wartość koszyka.")
    print(f"Wzrost: +{mean_diff:.0f} PLN ({mean_diff/koszyk_A.mean()*100:.1f}%) — przedział [{ci_low:.0f}, {ci_high:.0f}] PLN")
else:
    print(f"WYNIK: p ≥ 0.05 → Brak podstaw do odrzucenia H₀")
    print("WNIOSEK: Brak statystycznych dowodów na różnicę między wersjami.")

# Wizualizacja
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].hist(koszyk_A, bins=20, alpha=0.6, color='steelblue', density=True,
             label=f'Wersja A (x̄={koszyk_A.mean():.0f} PLN)')
axes[0].hist(koszyk_B, bins=20, alpha=0.6, color='tomato', density=True,
             label=f'Wersja B (x̄={koszyk_B.mean():.0f} PLN)')
axes[0].axvline(koszyk_A.mean(), color='steelblue', linestyle='--', linewidth=2)
axes[0].axvline(koszyk_B.mean(), color='tomato', linestyle='--', linewidth=2)
axes[0].set_title(f'Wartość koszyka A vs B\np={p_val:.4f}')
axes[0].set_xlabel('Koszyk (PLN)')
axes[0].set_ylabel('Gęstość')
axes[0].legend()

axes[1].boxplot([koszyk_A, koszyk_B], labels=['Wersja A', 'Wersja B'],
                patch_artist=True,
                boxprops=dict(facecolor='lightblue', color='steelblue'),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_title('Boxplot — porównanie grup')
axes[1].set_ylabel('Koszyk (PLN)')
axes[1].grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()

### 3.3 Sparowany t-test — efekt szkolenia sprzedażowego

In [ ]:
np.random.seed(42)

# SCENARIUSZ: 30 handlowców — sprzedaż przed i po szkoleniu
# To ci SAMI ludzie → dane powiązane → sparowany t-test
sprzedaz_przed = np.random.normal(loc=42000, scale=8000, size=30)
sprzedaz_po    = sprzedaz_przed * np.random.normal(1.15, 0.05, 30)

print("=" * 55)
print("Sparowany t-test — szkolenie sprzedażowe")
print("=" * 55)
print(f"H₀: μ_różnica = 0 (szkolenie nie zmienia sprzedaży)")
print(f"H₁: μ_różnica ≠ 0")
print()
print(f"Przed: x̄ = {sprzedaz_przed.mean():.0f} PLN/mies")
print(f"Po:    x̄ = {sprzedaz_po.mean():.0f} PLN/mies")
roznice = sprzedaz_po - sprzedaz_przed
print(f"Zmiana: +{roznice.mean():.0f} PLN ({roznice.mean()/sprzedaz_przed.mean()*100:.1f}%)")
print()

t_stat, p_val = stats.ttest_rel(sprzedaz_przed, sprzedaz_po)

print(f"Statystyka t = {t_stat:.4f}")
print(f"p-wartość    = {p_val:.2e}")
print()
if p_val < 0.05:
    roczny_wzrost = roznice.mean() * 12 * 30
    print("WYNIK: p < 0.05 → Szkolenie STATYSTYCZNIE ISTOTNIE zwiększa sprzedaż.")
    print(f"Roczny wzrost sprzedaży (30 handlowców): +{roczny_wzrost:,.0f} PLN")

---
## 4. Pełna analiza A/B testu — kampania e-mailowa

**5 kroków analizy A/B testu:**
1. Statystyki opisowe
2. Test normalności
3. Test właściwy (t-test)
4. Przedział ufności dla różnicy
5. Wniosek biznesowy

In [ ]:
np.random.seed(42)

# SCENARIUSZ: Sklep internetowy
# Wersja A: standardowy email | Wersja B: email z personalizacją
# Metryka: wartość zakupu po kliknięciu (PLN)

n_A = 150
n_B = 150

kampania_A = np.concatenate([
    np.random.normal(320, 80, 130),
    np.random.normal(800, 100, 20)
])
np.random.shuffle(kampania_A)

kampania_B = np.concatenate([
    np.random.normal(360, 85, 130),
    np.random.normal(850, 100, 20)
])
np.random.shuffle(kampania_B)

print("=" * 55)
print("ANALIZA A/B TESTU — Kampania e-mailowa")
print("=" * 55)
print()

# KROK 1: Statystyki opisowe
print("KROK 1: Statystyki opisowe")
print(f"  Wersja A: n={n_A}, x̄={kampania_A.mean():.2f} PLN, "
      f"mediana={np.median(kampania_A):.2f}, s={kampania_A.std():.2f}")
print(f"  Wersja B: n={n_B}, x̄={kampania_B.mean():.2f} PLN, "
      f"mediana={np.median(kampania_B):.2f}, s={kampania_B.std():.2f}")
print(f"  Różnica: {kampania_B.mean()-kampania_A.mean():.2f} PLN "
      f"({(kampania_B.mean()/kampania_A.mean()-1)*100:.1f}%)")
print()

# KROK 2: Normalność — podpróba 50
print("KROK 2: Test normalności (Shapiro-Wilk, podpróba 50 obs.)")
np.random.seed(42)
idx = np.random.choice(len(kampania_A), 50, replace=False)
stat_A, p_A = stats.shapiro(kampania_A[idx])
stat_B, p_B = stats.shapiro(kampania_B[idx])
print(f"  Wersja A: W={stat_A:.4f}, p={p_A:.4f} {'(normalny)' if p_A > 0.05 else '(nie-normalny)'}")
print(f"  Wersja B: W={stat_B:.4f}, p={p_B:.4f} {'(normalny)' if p_B > 0.05 else '(nie-normalny)'}")
print("  Uwaga: mieszana dystrybucja (premium + standardowi) może odbiegać od normalnej.")
print("  Przy n=150 t-test jest odporny (Central Limit Theorem).")
print()

# KROK 3: Welch's t-test
print("KROK 3: Welch's t-test")
t_stat, p_val = stats.ttest_ind(kampania_A, kampania_B, equal_var=False)
print(f"  t = {t_stat:.4f}, p = {p_val:.4f}")
print()

# KROK 4: Przedział ufności dla różnicy
print("KROK 4: 95% CI dla różnicy (B–A)")
mean_diff = kampania_B.mean() - kampania_A.mean()
se_diff = np.sqrt(kampania_A.var()/n_A + kampania_B.var()/n_B)
df_w = n_A + n_B - 2
ci_low = mean_diff - stats.t.ppf(0.975, df_w) * se_diff
ci_high = mean_diff + stats.t.ppf(0.975, df_w) * se_diff
print(f"  Różnica = {mean_diff:.2f} PLN")
print(f"  95% CI  = [{ci_low:.2f}, {ci_high:.2f}] PLN")
print()

# KROK 5: Wniosek biznesowy
print("KROK 5: WNIOSEK BIZNESOWY")
print("-" * 45)
if p_val < 0.05 and ci_low > 0:
    print(f"  Personalizacja e-maila ISTOTNIE zwiększa wartość zakupu.")
    print(f"  Wzrost: {mean_diff:.0f} PLN/klienta (95% CI: [{ci_low:.0f}, {ci_high:.0f}] PLN).")
    print(f"  Przy 10 000 klientach/mies. → +{mean_diff*10000:,.0f} PLN/mies.")
    print(f"  REKOMENDACJA: Wdrożyć wersję B jako standard kampanii.")
else:
    print(f"  Brak statystycznych dowodów na przewagę personalizacji.")
    print(f"  REKOMENDACJA: Nie wdrażać, rozważyć inne optymalizacje.")

In [ ]:
# Wizualizacja wyników A/B testu — 4 panele
fig, axes = plt.subplots(2, 2, figsize=(13, 9), constrained_layout=True)

# Panel 1: Histogramy
axes[0, 0].hist(kampania_A, bins=25, alpha=0.6, color='steelblue', density=True,
                label=f'Wersja A (x̄={kampania_A.mean():.0f} PLN)')
axes[0, 0].hist(kampania_B, bins=25, alpha=0.6, color='tomato', density=True,
                label=f'Wersja B (x̄={kampania_B.mean():.0f} PLN)')
axes[0, 0].axvline(kampania_A.mean(), color='steelblue', linestyle='--', lw=2)
axes[0, 0].axvline(kampania_B.mean(), color='tomato', linestyle='--', lw=2)
axes[0, 0].set_title(f'Rozkład wartości zakupu\nt={t_stat:.2f}, p={p_val:.4f}', fontsize=11)
axes[0, 0].set_xlabel('Wartość zakupu (PLN)')
axes[0, 0].set_ylabel('Gęstość')
axes[0, 0].legend(fontsize=9)

# Panel 2: Boxplot
bp = axes[0, 1].boxplot(
    [kampania_A, kampania_B],
    labels=['Wersja A\n(standardowa)', 'Wersja B\n(personalizowana)'],
    patch_artist=True,
    boxprops=dict(facecolor='lightblue'),
    medianprops=dict(color='red', linewidth=2)
)
bp['boxes'][1].set_facecolor('lightsalmon')
axes[0, 1].set_title('Boxplot — porównanie rozkładów', fontsize=11)
axes[0, 1].set_ylabel('Wartość zakupu (PLN)')
axes[0, 1].grid(True, axis='y', alpha=0.3)

# Panel 3: Przedział ufności
axes[1, 0].errorbar(
    x=['Różnica B–A'],
    y=[mean_diff],
    yerr=[[mean_diff - ci_low], [ci_high - mean_diff]],
    fmt='o', color='darkgreen', markersize=12, capsize=12, linewidth=2.5,
    label=f'Różnica = {mean_diff:.1f} PLN'
)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=1.5, label='H₀: różnica = 0')
axes[1, 0].set_title('95% CI dla różnicy (B–A)\nCały przedział > 0 → pewna korzyść', fontsize=11)
axes[1, 0].set_ylabel('Różnica wartości zakupu (PLN)')
axes[1, 0].legend(fontsize=9)
axes[1, 0].grid(True, axis='y', alpha=0.3)

# Panel 4: Efekt biznesowy
klienci = np.arange(1000, 15001, 1000)
dodatkowy = klienci * mean_diff
axes[1, 1].bar(klienci // 1000, dodatkowy / 1000,
               color='mediumseagreen', edgecolor='white')
axes[1, 1].set_title('Szacowany dodatkowy przychód\n(per liczba klientów/mies.)', fontsize=11)
axes[1, 1].set_xlabel('Klienci miesięcznie (tysiące)')
axes[1, 1].set_ylabel('Dodatkowy przychód (tys. PLN/mies.)')
axes[1, 1].grid(True, axis='y', alpha=0.3)

fig.suptitle('A/B Test Kampania e-mailowa — Wyniki pełnej analizy',
             fontsize=13, fontweight='bold')
plt.show()
plt.close()

---
## 5. Test chi-kwadrat — dane kategoryczne

**Kiedy używamy chi-kwadrat?**  
Gdy pytamy: *"czy dwie zmienne kategoryczne są od siebie niezależne?"*

T-test → porównuje **liczby** (średnie)  
Chi-kwadrat → porównuje **liczebności** (tabela krzyżowa)

In [ ]:
np.random.seed(42)

# SCENARIUSZ: Ankieta 300 klientów — czy wiek wpływa na wybór produktu?
dane_ankieta = np.array([
    [55, 35, 10],   # 18-35 lat: preferują Budżetowy
    [30, 50, 20],   # 36-55 lat: preferują Średni
    [15, 35, 50],   # 55+ lat:   preferują Premium
])

grupy    = ['18-35 lat', '36-55 lat', '55+ lat']
produkty = ['Budżetowy', 'Średni', 'Premium']

print("=" * 55)
print("Chi-kwadrat — preferencje produktowe wg wieku")
print("=" * 55)
print()
print("Tabela kontyngencji (obserwowane):")
print(pd.DataFrame(dane_ankieta, index=grupy, columns=produkty))
print()

chi2_stat, p_val, dof, expected = stats.chi2_contingency(dane_ankieta)

print(f"Chi² = {chi2_stat:.4f}")
print(f"df   = {dof}")
print(f"p    = {p_val:.6f}")
print()
print("Wartości oczekiwane (przy H₀: brak zależności):")
print(pd.DataFrame(expected.round(1), index=grupy, columns=produkty))
print()
if p_val < 0.05:
    print("WYNIK: p < 0.05 → Preferencje produktowe ZALEŻĄ od grupy wiekowej.")
    print("AKCJA: Segmentacja marketingowa wg wieku — personalizacja oferty!")

# Wizualizacja
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Stacked bar chart
colors = ['#3498db', '#2ecc71', '#e74c3c']
bottom = np.zeros(3)
for i, (prod, kolor) in enumerate(zip(produkty, colors)):
    axes[0].bar(grupy, dane_ankieta[:, i], bottom=bottom,
                label=prod, color=kolor, alpha=0.85, edgecolor='white')
    bottom += dane_ankieta[:, i]
axes[0].set_title('Preferencje produktowe wg grupy wiekowej', fontsize=11)
axes[0].set_xlabel('Grupa wiekowa')
axes[0].set_ylabel('Liczba klientów')
axes[0].legend(title='Produkt')

# Heatmapa residuów standaryzowanych
residua = (dane_ankieta - expected) / np.sqrt(expected)
im = axes[1].imshow(residua, cmap='RdBu_r', aspect='auto', vmin=-3, vmax=3)
axes[1].set_xticks(range(len(produkty)))
axes[1].set_yticks(range(len(grupy)))
axes[1].set_xticklabels(produkty)
axes[1].set_yticklabels(grupy)
for i in range(len(grupy)):
    for j in range(len(produkty)):
        axes[1].text(j, i, f'{residua[i,j]:.2f}', ha='center', va='center',
                     fontsize=11, fontweight='bold',
                     color='white' if abs(residua[i,j]) > 1.5 else 'black')
plt.colorbar(im, ax=axes[1], label='Residuum standaryzowane')
axes[1].set_title(f'Residua: czerwony = więcej niż oczekiwano\nχ²={chi2_stat:.1f}, p={p_val:.4f}', fontsize=11)

plt.tight_layout()
plt.show()
plt.close()

---
## 6. Przedziały ufności — pewność co do estymacji

In [ ]:
np.random.seed(42)

# SCENARIUSZ: Badamy NPS (Net Promoter Score) u 80 klientów
nps_scores = np.random.normal(loc=7.2, scale=1.8, size=80).clip(0, 10)

n        = len(nps_scores)
mean_nps = nps_scores.mean()
se       = stats.sem(nps_scores)

print("=" * 55)
print("Przedziały ufności dla średniego NPS")
print("=" * 55)
print(f"Próba: n={n}, x̄={mean_nps:.3f}, SE={se:.4f}")
print()
print(f"{'Poziom ufności':18} | {'Dolna':>8} | {'Górna':>8} | {'Szerokość':>10}")
print("-" * 55)

poziomy = [0.90, 0.95, 0.99]
ci_results = []
for poziom in poziomy:
    ci = stats.t.interval(poziom, df=n-1, loc=mean_nps, scale=se)
    szerokosc = ci[1] - ci[0]
    ci_results.append((poziom, ci[0], ci[1], szerokosc))
    print(f"{poziom*100:.0f}%              | {ci[0]:>8.3f} | {ci[1]:>8.3f} | {szerokosc:>10.3f}")

print()
ci_95 = ci_results[1]
print(f"Interpretacja 95% CI:")
print(f"  'Jesteśmy w 95% pewni, że prawdziwy średni NPS mieści się")
print(f"  między {ci_95[1]:.2f} a {ci_95[2]:.2f}'")
print()

cel = 7.5
print(f"Decyzja: Cel firmy to NPS > {cel}.")
if ci_95[2] < cel:
    print(f"  Górna granica CI ({ci_95[2]:.2f}) < {cel} → Cel NIE jest osiągnięty.")
elif ci_95[1] > cel:
    print(f"  Dolna granica CI ({ci_95[1]:.2f}) > {cel} → Cel jest osiągnięty.")
else:
    print(f"  CI obejmuje {cel} → wynik niejednoznaczny — potrzeba więcej danych.")

# Wizualizacja
fig, ax = plt.subplots(figsize=(9, 5))
colors_ci = ['#f39c12', '#2980b9', '#8e44ad']
for i, (poziom, ci_low, ci_high, szer) in enumerate(ci_results):
    ax.errorbar(
        x=[poziom], y=[mean_nps],
        yerr=[[mean_nps - ci_low], [ci_high - mean_nps]],
        fmt='o', color=colors_ci[i], markersize=10, capsize=12, linewidth=2.5,
        label=f'{poziom*100:.0f}% CI: [{ci_low:.2f}, {ci_high:.2f}]'
    )
ax.axhline(cel, color='red', linestyle='--', linewidth=1.5, label=f'Cel NPS = {cel}')
ax.axhline(mean_nps, color='black', linestyle='-', linewidth=1, alpha=0.4,
           label=f'Próbkowa x̄ = {mean_nps:.2f}')
ax.set_title('Przedziały ufności dla średniego NPS\n(im wyższy poziom ufności, tym szerszy przedział)', fontsize=11)
ax.set_xlabel('Poziom ufności')
ax.set_ylabel('Średni NPS')
ax.set_xticks(poziomy)
ax.set_xticklabels([f'{p*100:.0f}%' for p in poziomy])
ax.legend(fontsize=9, loc='lower right')
ax.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

---
## 7. Aktywność — A/B test reklamy wideo

**Zadanie:** Kampania w social media. Wersja A = grafika statyczna, Wersja B = reklama wideo.  
Metryka: czas spędzony na stronie produktu (sekundy).  
Pytanie: Czy reklama wideo zatrzymuje użytkowników dłużej?

In [ ]:
np.random.seed(42)
czas_A = np.random.normal(loc=45, scale=15, size=120)
czas_B = np.random.normal(loc=52, scale=18, size=120)

# KROK 1: Statystyki opisowe
print("KROK 1: Statystyki opisowe")
print(f"  Wersja A (grafika): n={len(czas_A)}, x̄={czas_A.mean():.1f}s, "
      f"med={np.median(czas_A):.1f}s, s={czas_A.std():.1f}s")
print(f"  Wersja B (wideo):   n={len(czas_B)}, x̄={czas_B.mean():.1f}s, "
      f"med={np.median(czas_B):.1f}s, s={czas_B.std():.1f}s")
print(f"  Różnica: {czas_B.mean()-czas_A.mean():.1f}s ({(czas_B.mean()/czas_A.mean()-1)*100:.1f}%)")
print()

# KROK 2: Normalność
print("KROK 2: Test normalności (podpróba 50)")
np.random.seed(42)
idx50 = np.random.choice(len(czas_A), 50, replace=False)
for nazwa, dane in [('Wersja A', czas_A), ('Wersja B', czas_B)]:
    s, p = stats.shapiro(dane[idx50])
    print(f"  {nazwa}: W={s:.4f}, p={p:.4f} ({'normalny' if p > 0.05 else 'nie-normalny'})")
print()

# KROK 3: Welch's t-test
print("KROK 3: Welch's t-test")
t_stat, p_val = stats.ttest_ind(czas_A, czas_B, equal_var=False)
print(f"  t = {t_stat:.4f}, p = {p_val:.6f}")
print()

# KROK 4: CI dla różnicy
print("KROK 4: 95% CI dla różnicy (B–A)")
diff = czas_B.mean() - czas_A.mean()
se_d = np.sqrt(czas_A.var()/len(czas_A) + czas_B.var()/len(czas_B))
df_d = len(czas_A) + len(czas_B) - 2
ci_lo = diff - stats.t.ppf(0.975, df_d) * se_d
ci_hi = diff + stats.t.ppf(0.975, df_d) * se_d
print(f"  Różnica = {diff:.2f}s, 95% CI = [{ci_lo:.2f}, {ci_hi:.2f}]s")
print()

# KROK 5: Wniosek biznesowy
print("KROK 5: WNIOSEK BIZNESOWY")
print("-" * 45)
if p_val < 0.05:
    print(f"  Reklama wideo STATYSTYCZNIE ISTOTNIE wydłuża czas na stronie.")
    print(f"  Wzrost: +{diff:.0f}s/sesję (95% CI: [{ci_lo:.0f}, {ci_hi:.0f}]s).")
    print(f"  REKOMENDACJA: Wdrożyć wersję wideo (B) jako standard kampanii.")
else:
    print(f"  Brak dowodów na przewagę reklamy wideo.")
    print(f"  REKOMENDACJA: Nie wdrażać, rozważyć inne formaty.")

---
## 8. Podsumowanie — ściągawka

### Kluczowe funkcje scipy.stats

```python
# Rozkład normalny
stats.norm.pdf(x, loc=mu, scale=sigma)     # gęstość
stats.norm.cdf(x, loc=mu, scale=sigma)     # P(X <= x)
stats.norm.ppf(p, loc=mu, scale=sigma)     # kwantyl (odwrotność CDF)

# Diagnostyka normalności
stat, p = stats.shapiro(dane)              # Shapiro-Wilk (n < 5000)
stats.probplot(dane, dist='norm', plot=ax) # QQ-plot wizualny

# T-testy
t, p = stats.ttest_1samp(dane, popmean=X)         # 1 próba vs wartość
t, p = stats.ttest_ind(A, B, equal_var=False)      # 2 niezależne (Welch)
t, p = stats.ttest_rel(przed, po)                  # 2 powiązane (sparowany)

# Chi-kwadrat
chi2, p, dof, exp = stats.chi2_contingency(tab)    # tabela = np.array 2D

# Przedziały ufności
se = stats.sem(dane)
ci = stats.t.interval(0.95, df=n-1, loc=mean, scale=se)
```

### Schemat 5-krokowej analizy A/B

```
1. Statystyki opisowe (n, x̄, mediana, s)
2. Test normalności (Shapiro + QQ-plot)
3. Test właściwy (t-test lub chi-kwadrat)
4. Przedział ufności dla różnicy
5. Wniosek biznesowy (w języku managera, nie statystyka)
```

### Interpretacja p-wartości

| p-wartość | Interpretacja |
|-----------|---------------|
| p < 0.001 | Bardzo silne dowody przeciw H₀ |
| p < 0.01  | Silne dowody przeciw H₀ |
| p < 0.05  | Umiarkowane dowody — odrzucamy H₀ (standard) |
| p ≥ 0.05  | Brak wystarczających dowodów — nie odrzucamy H₀ |

> **Pamiętaj:** Istotność statystyczna ≠ ważność biznesowa!